In [1]:
import configparser
import os
import pickle

import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from scipy.spatial.distance import cosine
from eval_tensor import VerbTensorEvaluator

In [2]:
import matplotlib.pyplot as plt
%pylab inline
pylab.rcParams['figure.figsize'] = (20, 10)

Populating the interactive namespace from numpy and matplotlib


In [3]:
config = configparser.ConfigParser()
config.read('config.ini')
tensor_dir = os.path.join(config['DEFAULT']['ProjectDirectory'], 'tensor/')

In [4]:
evalor = VerbTensorEvaluator(cutoff=50000, decomp_algo='parafac', rank=256)

In [5]:
evalor.load_embeddings()

In [6]:
subj = pd.Series(evalor.index['nsubj'])

In [7]:
obj = pd.Series(evalor.index['dobj'])

In [8]:
cutoff=80000

In [9]:
vocab = list(subj.head(cutoff).index & obj.head(cutoff).index)

/home/makrai/tool/python/vtensor/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  """Entry point for launching an IPython kernel.


In [10]:
subj_embed = evalor.decomped_tns.factors[0][np.array([subj.loc[w] for w in vocab])]

In [11]:
obj_embed = evalor.decomped_tns.factors[2][np.array([obj.loc[w] for w in vocab])]

In [12]:
#regression = LinearRegression(n_jobs=-1)

In [13]:
#regression.fit(subj_embed, obj_embed)

In [14]:
sim = np.sum(subj_embed*(obj_embed), axis=1)

In [15]:
#diff_norm = np.apply_along_axis(np.linalg.norm, 1, diff.todense())

In [16]:
', '.join([vocab[i] for i in np.argsort(sim.todense())[:10]])

'analysis, decision, diagram, insight, example, summary, show, overview, guide, introduction'

In [22]:
', '.join([vocab[i] for i in np.argsort(-sim.todense())[:10]])

', he, she, they, -rsb-, that, $, -rsb-_NNS, we, I'

In [18]:
noun = 'room'

In [19]:
df = pd.read_pickle(os.path.join(config['DEFAULT']['ProjectDirectory'], 'dataframe/assoc.pkl')).reset_index()

In [20]:
df[(df.nsubj!='')&(df.dobj!='')&((df.nsubj==noun)|(df.dobj==noun))].head(10)

,nsubj,ROOT,dobj,freq,freq_nsubj,freq_ROOT,freq_dobj,"freq_('nsubj', 'ROOT')","freq_('nsubj', 'dobj')","freq_('ROOT', 'dobj')",...,log_freq,log_prob,pmi,iact,npmi,niact,pmi_sali,iact_sali,ldice,ldice_sali
4052,we,have,room,267570,631539931,759840025,8169644,75910466,569581,1542706,...,18.029557,-15.537170,3.465750,0.000000,0.223062,0.000000,62.485945,0.000000,20.547688,370.465703
6312,I,have,room,178670,1669086425,759840025,8169644,138262188,512408,1542706,...,17.446938,-16.119789,1.481019,0.000000,0.091876,0.000000,25.839242,0.000000,19.164868,334.368265
7001,you,have,room,163517,1035768758,759840025,8169644,108966044,600674,1542706,...,17.319081,-16.247646,2.041519,0.289187,0.125650,0.017799,35.357227,5.008452,19.471152,337.222468
12257,they,have,room,99944,446678560,759840025,8169644,42480004,257583,1542706,...,16.608832,-16.957895,2.544663,0.000000,0.150058,0.000000,42.263880,0.000000,19.331340,321.070981
15232,it,have,room,82308,549081903,759840025,8169644,27477261,248687,1542706,...,16.328745,-17.237982,1.966791,0.000000,0.114096,0.000000,32.115233,0.000000,18.934483,309.176343
15584,you,reserve,room,80594,1035768758,6715100,8169644,184864,600674,199990,...,16.298385,-17.268342,7.842965,0.000000,0.454182,0.000000,127.827668,0.000000,19.230191,313.421058
21737,room,have,view,58900,4157889,759840025,9008517,786042,93286,1394671,...,15.845980,-17.720747,8.388036,0.000000,0.473346,0.000000,132.916653,0.000000,19.220521,304.567994
23160,room,have,bed,55617,4157889,759840025,1615143,786042,89718,560001,...,15.763238,-17.803489,10.784922,0.824145,0.605776,0.046291,170.005298,12.991191,19.151644,301.891935
24740,hotel,have,room,52261,2701996,759840025,8169644,345170,125226,1542706,...,15.673447,-17.893280,8.978345,0.000000,0.501772,0.000000,140.721621,0.000000,19.052277,298.614863
25603,room,have,bathroom,50581,4157889,759840025,635216,786042,88835,203506,...,15.626308,-17.940419,11.994335,0.832791,0.668565,0.046420,187.427165,13.013443,19.016562,297.158650


In [21]:
df.loc[:,:,'excellence'].head()

IndexingError: Too many indexers

In [ ]:
asymm = pd.Series(np.apply_along_axis(np.linalg.norm, 1, subj_embed - obj_embed),
                 index=vocab)

In [ ]:
#asymm.drop(np.nan, inplace=True)

In [ ]:
asymm.plot()#linewidth=1)
plt.yscale('log')

In [ ]:
asymm.head(1000).plot()#linewidth=1)
plt.yscale('log')

In [ ]:
asymm.head(200).plot()#linewidth=1)
plt.yscale('log')

In [ ]:
np.where(asymm>.5)

In [ ]:
asymm[132:142]

In [ ]:
index['nsubj']['year']

In [ ]:
index['nsubj']['place']

In [ ]:
index['nsubj']['staff']

In [ ]:
index['dobj']['year']

In [ ]:
index['dobj']['place']

In [ ]:
index['dobj']['staff']

In [ ]:
asymm[138]

In [ ]:
asymm.index[138]